# IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd

from date_time_preprocessor import *

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss, classification_report, roc_auc_score
import category_encoders as ce
import lightgbm as lgb

import pickle

# IMPORT DATA

In [2]:
path = '/Users/mitchell.carmen/Documents/FullStack_DS/'
df_raw = pd.read_csv(path + 'data/airline_delay_train.csv')
print(df_raw.shape)

(406045, 8)


In [3]:
# Applying date_time_preprocessor.py
training = feat_eng_datetime(df_raw)
print(training.shape)

Running feature engineering
(406045, 11)


In [4]:
training.head()

,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Day_of_Week,Year,Month,Day,Hour,Minutes
0,MQ,CVG,DFW,812,1,Sunday,2010,1,17,17,5
1,MQ,OMA,ORD,416,0,Friday,2010,1,29,17,3
2,US,SJC,PHX,622,0,Sunday,2010,1,31,18,3
3,YV,MTJ,DEN,197,0,Tuesday,2010,1,26,16,42
4,US,PHL,ORD,678,0,Wednesday,2010,1,6,17,53


In [6]:
X = training.drop(["dep_delayed_15min"], axis=1)
y = training["dep_delayed_15min"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    random_state=21,
    test_size=0.1,
    stratify=y)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(365440, 10)
(365440,)
(40605, 10)
(40605,)


# CREATE VAR TYPES

In [7]:
numerical_cols = list(X_train.select_dtypes(include=['int64']).columns)
categorical_cols = list(X_train.select_dtypes(include=['object', 'category']).columns)

# PRE PROCESSING PIPELINE

In [8]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value= -9999)),
    ('catencoder', ce.ordinal.OrdinalEncoder())])
#     ('targetencoder', ce.target_encoder.TargetEncoder(min_samples_leaf = 1, smoothing = 1))])
#     ('countencoder', ce.count.CountEncoder(min_group_size = 10))])
#     ('onehotencoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# MODEL BUILD PROTOTYPE

In [9]:
## hyperparameter tuning for Light GBM

LGBM = lgb.LGBMClassifier(boosting_type='gbdt',random_state = 99,
                          class_weight='balanced', objective= 'binary')

param_grid = { 
    'lgbmclassifier__n_estimators': [150,250],
    'lgbmclassifier__feature_fraction': ['auto', 'sqrt', 0.7],
    'lgbmclassifier__max_depth' : [6,7,8],
    'lgbmclassifier__learning_rate' : [0.1, 0.01],
    'lgbmclassifier__num_leaves' : [70,80],
    'lgbmclassifier__min_data_in_leaf' : [20, 50, 100]
}

lgbm = Pipeline(steps=[('preprocessor', preprocessor),
                      ('lgbmclassifier', LGBM)])

In [ ]:
grid_lgbm = GridSearchCV(lgbm, cv= 3, n_jobs= -1, param_grid= param_grid, scoring='roc_auc')
grid_lgbm.fit(X_train, y_train)

In [ ]:
grid_lgbm.best_score_

In [ ]:
grid_lgbm.best_params_

In [ ]:
y_pred = grid_lgbm.predict(X_test)

roc_auc_score(y_test,y_pred)

# SAVE PICKLE DUMP

In [ ]:
grid_lgbm.feature_names = list(X_train.columns.values)
pickle.dump(grid_lgbm, open(path + 'LGBM_flights_model.sav', 'wb'))

In [ ]:
############################################
############################################

# OPEN PICKLE DUMP

In [ ]:
grid_lgbm = pickle.load(open(path + 'LGBM_flights_model.sav', 'rb'))

In [ ]:
# Holdout Test

In [ ]:
# Import HO data
test_df = pd.read_csv(path + 'data/airline_delay_test.csv')
# Preprocess DateTime features
test_df = feat_eng_datetime(test_df)
test_df_y = test_df.dep_delayed_15min
test_df_x = test_df.drop(['dep_delayed_15min'], axis=1)

In [ ]:
# Test on HO data
test_pred = grid_lgbm.predict(test_df_x)

roc_auc_score(test_df_y,test_pred)